In [2]:
# default_exp agricultural_pricing_analytics

In [4]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

In [7]:
#export
import pandas as pd
from fastcore.script import *


In [6]:
#export
def not_empty(row):
    x = "".join(row)
    if len(x) > 0: 
        return True
    else:
        return False

In [7]:
#export
def remove_brand_row(line_groups):
    line_groups[0].remove('"Ministry of Industry,\n')
    return line_groups   

In [8]:
#export
def get_parishes(dct):
    parishes = []
    ps = dct[2]
    for i in ps:
        if len(i.strip()) > 5:
            parishes.append(i)
            
    return parishes
    

In [9]:
#export
def remove_empty_strings_from_data_rows(lst):
    new_lst = [] 
    for item in lst: 
        if len(item) > 0: 
            new_lst.append(item)
           
    return new_lst

def get_data(list_of_lines, number_of_parishes):
    data = []
   
    for line in list_of_lines:
        lst = line.split("|")
        l = len(lst)
        lst = remove_empty_strings_from_data_rows(lst)
        if len(lst) == 8 and number_of_parishes == 1:
            # then pad the list
            # remove the newline character at the end first
            lst.remove("\n")
            lst.extend(5*"-".split())
        try:
            lst.remove("-\n")
            lst.append("-")
        except:
            pass
        data.append(lst)
    return data
        
        

In [38]:
#export
def format_date(date):
    date = date.lower().replace(" ", "_").replace(",","")
    return date



august_07_2021


In [10]:
#export
def get_headers(list_of_lines):
    lst = []
    for line in list_of_lines:
        lst.append(line.split("|"))
    lst = flatten_list(lst)
    lst = remove_empty_strings_from_list(lst)
    lst_copy = lst.copy()
    commodity = lst_copy[12]
    variety_source = lst_copy[13] + " " + lst_copy[15]
    most_frequent = lst_copy[2] + " " + lst_copy[3]
    lst[:0] = [variety_source] 
    lst[:0] = [commodity]
    lst[:0] = ['parish']
    lst[:0] = ['date']
    lst.insert(6,most_frequent)
    del lst[7]
    del lst[7]
    lst.insert(11,most_frequent)
    del lst[12]
    del lst[12]
    # format the list
    lst = list(map(str.lower, lst))
    lst = list(map( lambda s: s.replace('"',''), lst ))
    return lst[:9]

In [11]:
#export
def clean_date(raw_date_string):
    clean_date_string = (raw_date_string
                         .replace("Week","")
                         .replace("Ending","")
                         .replace("|", "")
                         .replace('"',"")).rstrip()
    return clean_date_string.lstrip()

s = clean_date('Week Ending September 04, 2021"||||||||||||||||\n')
print(s)

September 04, 2021


In [28]:
#export
class ParishData:
   
    def __init__(self, parish, data, header):
        self.parish = parish.replace(" ","").lower()
        self.data = data
        self.header = header
        
    def write_to_csv(self,output_directory:str):
        import pandas as pd
        import os
        try:
            os.mkdir(f"{output_directory}/{report_date}") 
        except OSError as error: 
            print(f"{output_directory}/{report_date} already exists,not recreating")      

        self.data.insert(0,self.header)
        df = pd.DataFrame(self.data)
        df.to_csv(f"{output_directory}/{report_date}/{self.parish}_{report_date}.csv",sep="|" ,index=False, header=False)
        
        
    
    def __repr__(self):
        return f"{self.parish}: {self.data[:12]}" 

In [18]:
#export
def flatten_list(lst):
    flat_list = [item for sublist in lst for item in sublist] 
    return flat_list 

def remove_empty_strings_from_list(lst):
    _list = ' '.join(lst).split()
    return _list 

def create_parish_data(data_list,data_date,header,parish):
    result = ParishData(parish, data_list, header)    
    return result




{0: 'St.Thomas'}: [[1, 2, 3, 4], [5, 6, 7, 8]]


In [1]:
#export
def split_data(data_list,idx,parish_list,date):
    idx = 7
    r = []
    p0_data_list = []
    p1_data_list = []
    for d in data_list: 
        d = list(map( lambda s: s.replace('\n',''), d ))
        d = list(map(str.lower, d))
        if len(parish_list) == 2:
            p1 = parish_list[0]
            p2 = parish_list[1]
            d1 = d[:idx]
            d1[:0] = [p1]
            d1[:0] = [date]
            d2 = d[idx:]
            # let add the variety
            d2[:0] = [d[1]]
            # and the commodity
            d2[:0] = [d[0]]
            d2[:0] =  [p2]
            d2[:0] = [date]
            p0_data_list.extend([d1])
            p1_data_list.extend([d2])
            x = f"{len(p0_data_list[0])}, {p0_data_list}",f"{len(p1_data_list[0])}, {p1_data_list}"
            
        else: 
            p1 = parish_list[0]
            d1 = d[:idx]
            d1[:0] = [p1]
            d1[:0] = [date]
            p0_data_list.extend([d1])
    if len(parish_list) == 2:
        result = p0_data_list, p1_data_list
    else: 
        result = p0_data_list
        
    return  result
        


In [20]:
#export
def process_group(group,header):
        data_group_list = []
        result = []
        parishes_no_space  = {}
        idx = 0
        
        parish_list = ["Manchester",
                       "St.Andrew",
                       "St.Catherine",
                       "Clarendon",
                       "St.Elizabeth",
                       "Westmoreland",
                       "Hanover",
                       "St.James",
                       "Trelawny",
                       "St.Ann",
                       "St.Mary",
                       "Portland",
                       "St.Thomas"]   
        parishes = group[0]
        parishes = parishes.split("|")
        for p in parishes: 
            if len(p) > 5:
                parishes_no_space[idx]  = p
                idx = idx + 1
        # there is space between the manchester and st.andrew heading
        # and thier data, this space is not between the other headers 
        # and thier data :-) 
        offset =  5 if len(group) == 55 else 3        
        
        data_list = get_data(group[offset:],len(parishes_no_space)) 
             
        
        if len(parishes_no_space) == 2: 
            p1 = parishes_no_space[0]
            p2 = parishes_no_space[1]
            # split the data
            p1_data, p2_data = split_data(data_list,idx,[p1,p2],report_date)
            parish_data_1 = create_parish_data(p1_data,report_date , header,p1)
            parish_data_2 = create_parish_data(p2_data,report_date , header,p2)
            result.extend([parish_data_1,parish_data_2])
        else:
            # we only have one parish 
            p1 = parishes_no_space[0]
            p1_data = split_data(data_list,idx,[p1],report_date)
            parish_data_1 = create_parish_data( p1_data ,report_date, header,p1)
            result.extend([parish_data_1])
        
        return result
    
group = []  
header = ['date', 'parish', 'Commodity', 'Variety Source', 
'Low', 'High', '"Most Frequent"', 'Supply', 'Grade', 'Low', 
'High', '"Most Frequent"', 'Supply', 'Grade']

In [31]:
#export
global report_date  

In [39]:
#export
def run(input_path:str, output_directory:str):
    global report_date
    path = input_path
    with open(path) as file:
        groups =[]
        processed_groups = []
        current_group =[]
        
        group_boundary_str = "Prepared on"
        doc_lines = file.readlines()
        for line in doc_lines:
            if "Ministry of Industry" in line:
                continue
            if "Fisheries" in line:
                continue
            if "Commerce, Agriculture" in line:
                continue    
            if "Prepared on" in line:
                groups.append(current_group)
                current_group = []
                continue
            if len(line.replace("|", "").strip()) == 0:
                continue 
            if  "Week Ending" in line:
                report_date = clean_date(line)
                report_date =format_date(report_date)
                continue
            current_group.append(line)
    
    headers = get_headers(groups[0][1:5])
        
    for group in groups:
        group = process_group(group, headers)
        processed_groups.append(group)
        
    flat_list = [item for sublist in processed_groups for item in sublist]    
    for p in flat_list:
        p.write_to_csv(output_directory)  

In [26]:
#export
@call_parse
def main(input_path:Param("The farmgate file path", str),
         output_folder:Param("where the converted file should be written", str),
      ):
    "Clean up the input file and generates individual parish farm gate prices in the specified output folder"
    print(f"input path:{input_path}")
    run(input_path, output_folder)

In [42]:
notebook2script()


Converted farmgate_data_viz.ipynb.
Converted index.ipynb.
